In [1]:
import tensorflow as tf
import keras

2024-01-02 11:35:51.746683: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 11:35:51.746708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 11:35:51.747732: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-02 11:35:51.753570: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-02 11:35:52.411848: W tensorflow/compiler/tf2

In [2]:
def residual_block(inputs, filter,block_num,reduce=False):
    res = inputs
    x = keras.layers.Conv2D(filter,(3, 3), strides = 2 if reduce else 1,activation = 'relu', padding = 'same',name = f'conv_{block_num}_1')(res)
    x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_2')(x)
    if reduce == True:
        res = keras.layers.Conv2D(filter,(1, 1), strides = 2, activation = 'relu', padding = 'same',name = f'conv_skip_{block_num-1}')(res)
    res = keras.layers.Add(name = f'end_res_bock_{2*block_num-1}')([x,res])
    x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_3')(res)
    x = keras.layers.Conv2D(filter,(3, 3), activation = 'relu', padding = 'same',name = f'conv_{block_num}_4')(x)
    res = keras.layers.Add(name = f'end_res_bock_{2*block_num}')([x,res])
    return res

def resnet18(num_outputs=1):
    # Inputs
    inputs = keras.layers.Input(shape=(224,224,3),name = 'input')
    x = keras.layers.Conv2D(64,(7, 7),strides = 2, activation = 'relu', padding = 'same',name = 'conv2D')(inputs)
    x = keras.layers.MaxPooling2D(pool_size = (3,3),strides = 2 ,padding = 'same',name = 'max_pool_1')(x)
    # Block 1
    x = residual_block(x,64,1,False)
    # Block 2
    x = residual_block(x,128,2,True)
    # Block 3
    x = residual_block(x,256,3,True)
    # Block 4
    x = residual_block(x,512,4,True)
    # Ouputs
    x = keras.layers.GlobalAveragePooling2D(name = 'global_pooling')(x)
    if num_outputs==1:
        outputs = keras.layers.Dense(num_outputs,activation = 'sigmoid',name = 'outputs_sigmoid')(x)
    else:
        outputs = keras.layers.Dense(num_outputs,activation = 'softmax',name = 'outputs_softmax')(x)
    model = keras.Model(inputs = inputs, outputs = outputs, name = 'resnet18')
    return model

In [3]:
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
train_gen = img_gen.flow_from_directory('./data',class_mode="binary",target_size=(224,224))

Found 24959 images belonging to 2 classes.


In [4]:
model = resnet18(1)
optimizer = keras.optimizers.Adam()
loss_fn = keras.losses.BinaryCrossentropy()
train_acc_metric = keras.metrics.BinaryAccuracy()

2024-01-02 11:36:01.532910: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-02 11:36:01.568483: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-02 11:36:01.568685: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
epochs = 3
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_gen):
        y_batch_train = y_batch_train.reshape(-1,1)
        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:
            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = model(x_batch_train, training=True)  # Logits for this minibatch
            # Compute the loss value for this minibatch.
            loss_value = loss_fn(y_batch_train, logits)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, model.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * 32))
        if step==len(train_gen)-1:
            break


Start of epoch 0


2024-01-02 11:36:06.717168: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-02 11:36:06.800978: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-02 11:36:07.011989: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-02 11:36:09.932329: I external/local_xla/xla/service/service.cc:168] XLA service 0x1cb9d550 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-02 11:36:09.932348: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2070, Compute Capability 7.5
2024-01-02 11:36:09.936522: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1704170170.025309  510357 device_compiler.h:18

Training loss (for one batch) at step 0: 0.6939
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.6813
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.6899
Seen so far: 12832 samples
Training loss (for one batch) at step 600: 0.6833
Seen so far: 19232 samples


/home/ailab/anaconda3/envs/tan/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:868: UserWarning: Truncated File Read
  warnings.warn(str(msg))



Start of epoch 1
Training loss (for one batch) at step 0: 0.6863
Seen so far: 32 samples


In [25]:
model.evaluate(train_gen)

780/780 [==============================] - 30s 38ms/step - loss: 0.2175 - acc: 0.9075


[0.2175300270318985, 0.9074882864952087]

In [6]:
model.evaluate(train_gen)

2023-12-26 14:24:06.696502: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-26 14:24:06.786929: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-26 14:24:07.317086: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


554/780 [====================>.........] - ETA: 8s - loss: 0.6963 - binary_accuracy: 0.5003

/home/ailab/anaconda3/envs/tan/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:868: UserWarning: Truncated File Read
  warnings.warn(str(msg))


780/780 [==============================] - 32s 39ms/step - loss: 0.6963 - binary_accuracy: 0.5004


[0.6962601542472839, 0.5004206895828247]

In [5]:
model.compile(optimizer=optimizer, loss= loss_fn,metrics=keras.metrics.BinaryAccuracy())

In [6]:
model.fit(train_gen,epochs=10)

Epoch 1/10


2023-12-26 13:53:00.959289: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-26 13:53:01.049756: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-26 13:53:01.591411: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-26 13:53:02.422421: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fd479a47610 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-26 13:53:02.422443: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2070, Compute Capability 7.5
2023-12-26 13:53:02.427321: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1703573582.507264  198095 device_compiler.

368/780 [=============>................] - ETA: 25s - loss: 0.7403 - binary_accuracy: 0.5223

/home/ailab/anaconda3/envs/tan/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:868: UserWarning: Truncated File Read
  warnings.warn(str(msg))


780/780 [==============================] - 54s 60ms/step - loss: 0.7137 - binary_accuracy: 0.5302
Epoch 2/10
780/780 [==============================] - 45s 58ms/step - loss: 0.6815 - binary_accuracy: 0.5654
Epoch 3/10
780/780 [==============================] - 45s 58ms/step - loss: 0.6693 - binary_accuracy: 0.5898
Epoch 4/10
780/780 [==============================] - 45s 58ms/step - loss: 0.6419 - binary_accuracy: 0.6279
Epoch 5/10
780/780 [==============================] - 45s 58ms/step - loss: 0.6030 - binary_accuracy: 0.6736
Epoch 6/10
780/780 [==============================] - 45s 58ms/step - loss: 0.5503 - binary_accuracy: 0.7209
Epoch 7/10
780/780 [==============================] - 45s 58ms/step - loss: 0.4902 - binary_accuracy: 0.7636
Epoch 8/10
780/780 [==============================] - 45s 58ms/step - loss: 0.4441 - binary_accuracy: 0.7904
Epoch 9/10
780/780 [==============================] - 45s 58ms/step - loss: 0.3980 - binary_accuracy: 0.8204
Epoch 10/10
780/780 [=========

## Speedup with tf.function

In [7]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    train_acc_metric.update_state(y, logits)
    return loss_value,grads
@tf.function
def optimize_model(grads):
    optimizer.apply_gradients(zip(grads, model.trainable_weights))


In [9]:
epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_gen):
        y_batch_train = y_batch_train.reshape(-1,1)
        loss_value,grads = train_step(x_batch_train,y_batch_train)
        optimize_model(grads)
        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * 32))
        if step==len(train_gen)-1:
            break
    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()


Start of epoch 0
Training loss (for one batch) at step 0: 0.5041
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.3931
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.3361
Seen so far: 12832 samples
Training loss (for one batch) at step 600: 0.4429
Seen so far: 19232 samples
Training acc over epoch: 0.8048

Start of epoch 1
Training loss (for one batch) at step 0: 0.2352
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.6619
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.2736
Seen so far: 12832 samples
Training loss (for one batch) at step 600: 0.3332
Seen so far: 19232 samples
Training acc over epoch: 0.8372


In [11]:
epochs = 7
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_gen):
        y_batch_train = y_batch_train.reshape(-1,1)
        loss_value = train_step(x_batch_train,y_batch_train)
        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * 32))
        if step==len(train_gen)-1:
            break
    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()


Start of epoch 0
Training loss (for one batch) at step 0: 0.5662
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.6559
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.6046
Seen so far: 12832 samples
Training loss (for one batch) at step 600: 0.4874
Seen so far: 19232 samples
Training acc over epoch: 0.7032

Start of epoch 1
Training loss (for one batch) at step 0: 0.5174
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.5484
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.5752
Seen so far: 12832 samples
Training loss (for one batch) at step 600: 0.4768
Seen so far: 19232 samples
Training acc over epoch: 0.7483

Start of epoch 2
Training loss (for one batch) at step 0: 0.4044
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.4493
Seen so far: 6432 samples
Training loss (for one batch) at step 400: 0.3160
Seen so far: 12832 samples
Training loss (for one batch) at step 600: 0.5423
See